## Results 

In [ ]:
import os
import pandas as pd

ruta_carpeta = './results'
dfs = []

for carpeta_raiz, subdirs, archivos in os.walk(ruta_carpeta):
    for archivo in archivos:
        if archivo.lower().endswith('.csv'):
            ruta = os.path.join(carpeta_raiz, archivo)
            try:
                df = pd.read_csv(ruta)
                ruta_rel = os.path.relpath(ruta, ruta_carpeta)
                df['Fuente'] = ruta_rel
                dfs.append(df)
            except Exception as e:
                print(f"Error leyendo {ruta}: {e}")

if dfs:
    df_todos = pd.concat(dfs, ignore_index=True)
    print("Concatenación completada. Total filas:", len(df_todos))
else:
    print("No se encontraron CSVs válidos en o debajo de", ruta_carpeta)


Concatenación completada. Total filas: 48


In [9]:
import os
import pandas as pd
import numpy as np

METRICAS = ['SMAPE', 'MAE', 'MSE']

def load_results(filename):
    df = pd.read_csv(filename)
    if 'Intervalos Acumulados' not in df.columns:
        raise KeyError(f"No existe la columna 'Intervalos Acumulados' en {filename}")
    extr = df['Intervalos Acumulados'].str.extract(r'a\s*(\d+)')
    if extr.isna().any().any():
        print(f"Advertencia: en {filename}, algunas filas no siguen el formato esperado.")
    # Convertir a int, permitiendo NaN:
    df['Intervalo'] = extr.astype(float).astype('Int64')
    df.set_index('Intervalo', inplace=True)
    return df

def extraer_metricas(df, metricas=METRICAS):
    existentes = [m for m in metricas if m in df.columns]
    faltantes = [m for m in metricas if m not in df.columns]
    if faltantes:
        print(f"Columnas faltantes ({faltantes}); se rellenarán con NaN.")
    for met in faltantes:
        df[met] = np.nan
    # Devolver solo en el orden de metricas
    return df[metricas]

def find_file_in_tree(base_dir, target_filename):
    for root, dirs, files in os.walk(base_dir):
        if target_filename in files:
            return os.path.join(root, target_filename)
    return None

def obtener_tabla(horizonte, base_results_dir='./results'):
    tablas = {}
    lstm_configs = {
        'LSTM-7': 7,
        'LSTM-14': 14,
        'LSTM-21': 21
    }
    for modelo, ventana in lstm_configs.items():
        filename = f"lstm_{ventana}_{horizonte}.csv"
        ruta_encontrada = find_file_in_tree(base_results_dir, filename)
        if ruta_encontrada:
            df = load_results(ruta_encontrada)
            tablas[modelo] = extraer_metricas(df, METRICAS)
        else:
            raise FileNotFoundError(f"Archivo no encontrado: buscado '{filename}' en '{base_results_dir}' y subcarpetas.")
    tabla_resumen = pd.concat(tablas, axis=1)
    tabla_resumen.index.name = 'Intervalo'
    return tabla_resumen

def stack_table(tabla):
    tabla_apilada = tabla.stack(level=1)
    tabla_apilada.index.names = ['Intervalo', 'Métrica']
    return tabla_apilada

# Funciones de estilo, mismas que tenías
import numpy as _np
def highlight_min_max(row):
    min_val = row.min()
    max_val = row.max()
    estilos = []
    for v in row:
        # Usar isclose para floats
        if _np.isclose(v, min_val) and _np.isclose(v, max_val):
            estilos.append("")
        elif _np.isclose(v, min_val):
            estilos.append("background-color: blue; color: white")
        elif _np.isclose(v, max_val):
            estilos.append("background-color: red; color: white")
        else:
            estilos.append("")
    return estilos

def style_stacked(tabla, horizonte):
    return (tabla.style
            .format("{:.2f}")
            .apply(highlight_min_max, axis=1)
            .set_caption(f"Comparación de Modelos por Métrica (mínimo azul, máximo rojo) - horizonte {horizonte}"))


In [10]:
# Por ejemplo, horizonte 15:
try:
    tabla_15 = obtener_tabla(15, base_results_dir='./results')
    tabla_15_apil = stack_table(tabla_15)
    display(style_stacked(tabla_15_apil, '15'))
    tabla_15_apil.to_csv('tabla_apilada_horizonte_15.csv')
except FileNotFoundError as e:
    print(e)

# Horizonte 30:
try:
    tabla_30 = obtener_tabla(30, base_results_dir='./results')
    tabla_30_apil = stack_table(tabla_30)
    display(style_stacked(tabla_30_apil, '30'))
    tabla_30_apil.to_csv('tabla_apilada_horizonte_30.csv')
except FileNotFoundError as e:
    print(e)


C:\Users\criju\AppData\Local\Temp\ipykernel_7924\47754030.py:55: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  tabla_apilada = tabla.stack(level=1)


C:\Users\criju\AppData\Local\Temp\ipykernel_7924\47754030.py:55: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  tabla_apilada = tabla.stack(level=1)
